Importacion de librerias, datos y parámetros

In [ ]:
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
!mkdir speechcommands
!tar -xf speech_commands_v0.01.tar.gz -C /content/speechcommands

--2020-06-26 14:10:48--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.142.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz’

speech_commands_v0. 100%[===================>]   1.39G  93.8MB/s    in 16s     

2020-06-26 14:11:04 (89.6 MB/s) - ‘speech_commands_v0.01.tar.gz’ saved [1489096277/1489096277]



In [ ]:
import os
import glob
import librosa
import numpy as np
import pandas as pd
from google.colab import drive

In [ ]:
working_directory = '/content/speechcommands'
numbers = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Función suplementaria

In [ ]:
def calculate_features(filename, n_mfcc=12,delta=True,deltadelta=True,energy=True, summary_fn = [np.mean, np.std], summary_names=['mean','std']):
  #Abro el archivo:
  x, sr = librosa.core.load(filename,sr=None)
  
  #Calculo MFCCs
  features = librosa.feature.mfcc(x,sr=sr,n_mfcc=n_mfcc)
  feat_names = ['mfcc_{}'.format(i) for i in range(n_mfcc)]
  #Calculo energia:
  if energy:
    energy = librosa.feature.rmse(x)
    features = np.concatenate([features,energy])
    feat_names = feat_names + ['energy']
  #Aplico media y desvio estandar por defecto
  summary_features = np.concatenate([fn(features,axis=1) for fn in summary_fn])
  
  #Lo mismo con los delta
  if delta:
    deltafeatures = np.diff(features)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltafeatures,axis=1) for fn in summary_fn])])
    d_names = ['d{}'.format(name) for name in feat_names]
  else:
    d_names = []

  #Y con los delta de segundo orden
  if deltadelta:
    deltadeltafeatures = np.diff(features,n=2)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltadeltafeatures,axis=1) for fn in summary_fn])]) 
    dd_names = ['dd{}'.format(name) for name in feat_names]
  else:
    dd_names = []

  feat_names = feat_names + d_names + dd_names
  #feat_names = ['{}_{}'.format(feat,summary) for feat in feat_names for summary in summary_names]
  feat_names = ['{}_{}'.format(name_i,summ_i) for summ_i in summary_names for name_i in feat_names]
  return summary_features, feat_names  

#Division en Entrenamiento validación y prueba

In [ ]:
def fill_types(file_type, file, t, base_path = '/content/speechcommands'):
    for line in file:
        line = line.strip()
        for number in numbers:
            if line.startswith(number + '/'):
                file_type[os.path.join(base_path, line)] = t

In [ ]:
file_type = {}

with open('/content/speechcommands/testing_list.txt') as file:
    fill_types(file_type, file, 'testing', base_path = working_directory)

with open('/content/speechcommands/validation_list.txt') as file:
    fill_types(file_type, file, 'validation', base_path = working_directory)

# Generacion de dataset y guardado en drive

In [ ]:
data = []
columns = []

for number in numbers:
    for filepath in glob.glob(working_directory + '/' + number  + '/' + '/*.wav'):
        feat, names = calculate_features(filepath)
        t = file_type[filepath] if filepath in file_type else 'training'
        data.append(np.append(feat, [number, filepath, t]))
        columns = names + ['target', 'filename', 'type']


In [ ]:
dataset = pd.DataFrame(data, columns = columns)
print(dataset.shape)
dataset.to_csv('/content/drive/My Drive/aa-tp2-dataset.csv', header=True, index=True)

(23666, 81)
